In [1]:
import os
import cv2
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from tensorflow.keras.preprocessing import image
from tensorflow import keras
from tensorflow.keras.applications import resnet50, ResNet50
from tensorflow.keras.preprocessing import image


In [2]:
model = tf.keras.models.load_model(r'E:\AImodel\models\Face-ResNet-BicycleCrunch-multiLabel-model')


In [3]:
def preprocess_image(image_path, target_size=(128, 128)):
    """이미지 파일을 불러와 전처리하는 함수"""
    img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # 모델 예측을 위해 차원 확장
    img_array /= 255.0  # 이미지를 0과 1 사이로 스케일링
    return img_array

def load_and_preprocess_from_directory(directory_path, target_size=(128, 128)):
    """지정된 디렉토리 내의 모든 이미지를 불러와 전처리하는 함수"""
    processed_images = []
    for file_name in os.listdir(directory_path):
        file_path = os.path.join(directory_path, file_name)
        if file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            img = preprocess_image(file_path, target_size=target_size)
            processed_images.append(img)
    return np.vstack(processed_images)  # 전처리된 이미지들을 하나의 numpy 배열로 합침

dir = r'E:\AItemp\testdata\BC test02'
preprocessed_images = load_and_preprocess_from_directory(dir)


In [4]:
import pandas as pd
# 모델 예측 수행
model_predictions = model.predict(preprocessed_images)
# model_predictions는 [prediction1, prediction2, prediction3] 형태의 리스트를 반환할 것입니다.

# 슬라이딩 윈도우 함수 정의 (앞서 제시된 예시 사용)
def apply_sliding_window(sequence, window_size=5, method='mean'):
    smoothed_sequence = []
    for i in range(len(sequence)):
        start_idx = max(0, i - window_size // 2)
        end_idx = min(len(sequence), i + window_size // 2 + 1)
        window = sequence[start_idx:end_idx]
        if method == 'mean':
            smoothed_value = np.mean(window)
        elif method == 'median':
            smoothed_value = np.median(window)
        else:
            raise ValueError("Method should be 'mean' or 'median'")
        smoothed_sequence.append(smoothed_value)
    return smoothed_sequence



7/7 [==============================] - 3s 28ms/step


In [7]:
# 각 예측 결과에 대해 슬라이딩 윈도우 적용
smoothed_predictions = []
for prediction in model_predictions:
    smoothed = apply_sliding_window(prediction.flatten(), window_size=5, method='mean')  # 예측 결과가 2차원일 수 있으므로 flatten() 호출
    smoothed_predictions.append(smoothed)

# 슬라이딩 윈도우가 적용된 결과 확인
# 예: smoothed_predictions[0]는 조정된 prediction1의 결과를 포함합니다.
adjusted_first_prediction1 = smoothed_predictions[0]
adjusted_first_prediction2 = smoothed_predictions[1]
adjusted_first_prediction3 = smoothed_predictions[2]

# 조정된 첫 번째 예측 결과 분석 예시
# print("조정된 1 번째 예측 결과:", adjusted_first_prediction1)
# print("조정된 2 번째 예측 결과:", adjusted_first_prediction2)
# print("조정된 3 번째 예측 결과:", adjusted_first_prediction3)

# 조정된 예측 결과가 특정 임계값을 넘는지 체크하는 로직
threshold = 0.5  # 임계값 설정
exceeds_threshold1 = [pred > threshold for pred in adjusted_first_prediction1]
exceeds_threshold2 = [pred > threshold for pred in adjusted_first_prediction2]
exceeds_threshold3 = [pred > threshold for pred in adjusted_first_prediction3]



In [6]:
# 예측 결과와 임계값을 초과 여부를 데이터프레임으로 변환
df = pd.DataFrame({    
    '허리 지면 고정': exceeds_threshold1,
    '수축시 무릎 엉덩이 지남': exceeds_threshold2,
    '견갑골이 충분히 올라옴': exceeds_threshold3,
})

# 임계값을 초과하는지 여부를 True/False 대신에 예/아니오로 표시하고 싶다면,
# 다음과 같이 수정할 수 있습니다.
df = df.replace({True: '예', False: '아니오'})

# 결과 출력
print(df)


    허리 지면 고정 수축시 무릎 엉덩이 지남 견갑골이 충분히 올라옴
0          예           아니오          아니오
1          예           아니오          아니오
2          예           아니오          아니오
3          예           아니오          아니오
4          예           아니오          아니오
..       ...           ...          ...
205        예           아니오          아니오
206        예           아니오          아니오
207        예           아니오          아니오
208        예           아니오          아니오
209        예           아니오          아니오

[210 rows x 3 columns]
